<a href="https://colab.research.google.com/github/Angshu727/Angshu727/blob/main/AgroGenie_%F0%9F%8C%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌾 AgroGenie: Your AI-Powered Smart Farming Assistant

AgroGenie is an intelligent, multi-agent system built to empower farmers and agri-entrepreneurs with personalized crop planning, fertilizer guidance, and profit forecasting — all through a single AI-driven interface.

Designed to be practical, scalable, and region-aware, AgroGenie aligns with **UN Sustainable Development Goal 2: Zero Hunger**, helping improve agricultural outcomes through data-backed recommendations and smart decision-making.

## 🧠 What AgroGenie Can Do
Given basic inputs like **land size**, **soil type**, **region**, **crop**, and **budget**, AgroGenie can:
- 🌿 Suggest suitable crops based on climate, soil, and cost
- 💧 Recommend optimal fertilizer combinations (organic + inorganic)
- 📈 Estimate total yield, market price, and profit or loss
- 📃 Export a report and AI summary with recommendations for success

## 🔧 Powered By
This project is built using modern AI tools and agent coordination frameworks:
- 🧠 **Gemini Pro** – LLM for reasoning, planning, and natural language
- 🔁 **LangGraph** – for stateful multi-agent workflows
- 🧩 **LangChain** – to structure prompts and tools
- 🧪 **ipywidgets** – for building interactive dashboards in notebooks

## 🧬 Architecture Overview
AgroGenie uses a modular, multi-agent architecture:
- 📌 **Crop Advisor Agent** – recommends crops
- 💧 **Fertilizer Advisor Agent** – suggests fertilizer plans
- 📈 **Profit Estimator Agent** – estimates yield, pricing, and profit

Everything is orchestrated using a state machine (LangGraph) for clean control and flexible agent routing.

---



# 📦 Install Necessary Packages

This cell installs all the required Python packages for the AgroGenie project, including `langchain`, `langgraph`, `google-generativeai` (for Gemini), and `ipywidgets` (for the interactive user interface.

In [ ]:
!pip install -q langchain langgraph google-generativeai ipywidgets

# 🔐 Set Up Gemini API Key

This cell configures the Google Generative AI library with your Gemini API key.

**Instructions:**
1. Obtain your API key from [Google AI Studio](https://makersuite.google.com/app).
2. Replace `"your-api-key-here"` with your actual API key in the code below.

In [ ]:
import google.generativeai as genai

# 🔑 Add your API key here
genai.configure(api_key="your-api-key-here")

# Create a model instance
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash-latest",
    generation_config={"temperature": 0.7}
)

# 📋 Import Python Modules

This cell imports all the necessary modules from `langchain`, `langgraph`, and `ipywidgets` that are required to build the multi-agent system and its user interface.

In [ ]:
from langchain.tools import tool
from langgraph.graph import StateGraph, END
import ipywidgets as widgets
from IPython.display import display, clear_output

# 🌾 Tool: Crop Advisor Agent

This function simulates a Crop Advisor Agent using the Gemini model. It takes user input regarding soil, land size, region, and budget, and uses the language model to suggest 2-3 suitable and sustainable crops.

The function `crop_advisor_tool` takes a `user_input` dictionary and returns a dictionary containing a list of `crop_suggestions`.

In [ ]:
def crop_advisor_tool(user_input):
    prompt = f"""
You are a smart agriculture assistant.

Suggest 2–3 suitable crops for:
- Soil: {user_input['soil']}
- Land: {user_input['land']} acres
- Region: {user_input['region']}
- Budget: ₹{user_input['budget']}

Respond briefly in one sentence with crop names and why they're suitable.
"""
    response = model.generate_content(prompt)
    # Assuming the response is a comma-separated string of crop names, split it into a list
    crop_suggestions = [crop.strip() for crop in response.text.strip().split(',')]
    return {
        "crop_suggestions": crop_suggestions
    }

# 💧 Tool: Fertilizer Advisor Agent

This function simulates a Fertilizer Advisor Agent using the Gemini model. It takes user input about the selected crop, soil type, budget, and region to suggest an eco-friendly fertilizer plan.

The function `fertilizer_advisor_tool` takes a `user_input` dictionary and returns a dictionary containing a list of `fertilizer_plan` suggestions.

In [ ]:
def fertilizer_advisor_tool(user_input):
    prompt = f"""
You are a soil fertility advisor.

Suggest a suitable fertilizer plan for:
- Crop: {user_input['crop']}
- Soil: {user_input['soil']}
- Budget: ₹{user_input['budget']}
- Region: {user_input['region']}

Respond in one sentence with fertilizer names or organic methods.
"""
    response = model.generate_content(prompt)
    # Assuming the response is a comma-separated string of fertilizer names, split it into a list
    fertilizer_plan = [fert.strip() for fert in response.text.strip().split(',')]
    return {
        "fertilizer_plan": fertilizer_plan
    }

# 📈 Tool: Profit Estimator Agent

This function simulates a Profit Estimator Agent using the Gemini model. It takes user input and the current state of the graph (including crop and fertilizer suggestions) to estimate potential yield, market price, and total profit. It also provides comments and concerns about the input.

The function `profit_estimator_tool` takes the `user_input` dictionary and the full `full_state` of the graph and returns a dictionary containing the `ai_profit_analysis`, `estimated_yield`, `estimated_profit`, and `market_price`.

In [ ]:
import re

def extract_number(value):
    match = re.search(r"₹?([\d,]+)", value.replace(',', ''))
    return int(match.group(1)) if match else 0

def extract_yield(text):
    match = re.search(r"Total Estimated Yield.*?[:\-]\s*([\d,]+)\s*kg", text, re.IGNORECASE)
    return match.group(1).strip() + " kg" if match else "N/A"

def extract_price(text):
    matches = re.findall(r"\*\*?\s*([A-Za-z\s]+):\s*₹?(\d+)/kg", text)
    if matches:
        prices = [f"{crop.strip()}: ₹{price}/kg" for crop, price in matches]
        return ", ".join(prices)
    return "N/A"

def extract_profit(text):
    match = re.search(r"Estimated Profit.*?[:\-]\s*₹?([\d,]+)", text)
    if match:
        return f"₹{match.group(1).strip()}"
    else:
        # Try to find total revenue and cost for calculation fallback
        revenue_match = re.search(r"Total Revenue.*?=\s*₹?([\d,]+)", text)
        cost_match = re.search(r"Total Costs.*?=\s*₹?([\d,]+)", text)
        if revenue_match and cost_match:
            revenue = int(revenue_match.group(1).replace(',', ''))
            cost = int(cost_match.group(1).replace(',', ''))
            return f"₹{revenue - cost}"
    return "N/A"


def profit_estimator_tool(user_input, full_state):
    fertilizer_info = str(full_state.get('fertilizer_plan', 'Not available'))
    crop_info = str(full_state.get('crop_suggestions', 'Not available'))

    prompt = f"""
    You are an agriculture expert.

    Based on the following input:
    - Land: {user_input["land"]} square meters
    - Crop: {user_input['crop']}
    - Region: {user_input['region']}
    - Budget: ₹{user_input['budget']}
    - Fertilizer: {fertilizer_info}
    - Crop Suggestions: {crop_info}

    Estimate:
    1. Total yield (kg)
    2. Average market price (₹/kg)
    3. Total profit (₹)
    4. Any concerns about the input

    Answer in this format:
    - Estimated Yield: ...
    - Market Price: ...
    - Estimated Profit: ...
    - Comments: ...
    """

    response = model.generate_content(prompt)
    raw_text = response.text.strip()

    return {
      "ai_profit_analysis": raw_text,
      "estimated_yield": extract_yield(raw_text),
      "estimated_profit": extract_profit(raw_text),
      "market_price": extract_price(raw_text),
    }


# 📊 Define Graph State

This cell defines the `GraphState` using `TypedDict`. This structure represents the state that is passed between the nodes (agents) in the LangGraph workflow. It includes fields for user input, crop suggestions, fertilizer plan, estimated yield, estimated profit, and market price.

In [ ]:
from langgraph.graph import StateGraph
from typing import TypedDict, List, Dict, Any

class GraphState(TypedDict):
    """Represents the state of our graph.

    Attributes:
        input: User input data (land, soil, etc.)
        crop_suggestions: Suggested crops from CropAdvisor
        fertilizer_plan: Suggested fertilizer plan from FertilizerAdvisor
        estimated_yield: Estimated yield from ProfitEstimator
        estimated_profit: Estimated profit from ProfitEstimator
        fertilizer_budget_usage: Budget usage for fertilizers
    """
    input: Dict[str, Any]
    crop_suggestions: List[str] | None
    fertilizer_plan: List[str] | None
    estimated_yield: str | None
    estimated_profit: str | None
    fertilizer_budget_usage: str | None
    market_price: str | None
    ai_profit_analysis: str | None

# 🧠 Define LangGraph Nodes for Each Agent

This cell defines the individual nodes for the LangGraph. Each node wraps one of the agent tool functions (`crop_advisor_tool`, `fertilizer_advisor_tool`, `profit_estimator_tool`) and updates the `GraphState` with the results from that agent.

- `crop_node`: Executes the Crop Advisor Tool.
- `fertilizer_node`: Executes the Fertilizer Advisor Tool.
- `profit_node`: Executes the Profit Estimator Tool.

In [ ]:
from typing import TypedDict, List, Dict, Any
from langgraph.graph import StateGraph, END # Import StateGraph and END here as well, just in case they are needed

def crop_node(state: GraphState) -> GraphState:
    result = crop_advisor_tool(state["input"])
    state.update(result)
    return state

def fertilizer_node(state: GraphState) -> GraphState:
    result = fertilizer_advisor_tool(state["input"])
    state.update(result)
    return state

def profit_node(state: GraphState) -> GraphState:
    result = profit_estimator_tool(state["input"], state)
    state.update(result)
    return state

# 🔁 Construct LangGraph Multi-Agent Workflow

This cell builds the LangGraph, defining the sequence of execution for the agents. The workflow starts with the Crop Advisor, followed by the Fertilizer Advisor, and finally the Profit Estimator. The graph ends after the Profit Estimator node.

In [ ]:
builder = StateGraph(GraphState)

builder.add_node("CropAdvisor", crop_node)
builder.add_node("FertilizerAdvisor", fertilizer_node)
builder.add_node("ProfitEstimator", profit_node)


builder.set_entry_point("CropAdvisor")
builder.add_edge("CropAdvisor", "FertilizerAdvisor")
builder.add_edge("FertilizerAdvisor", "ProfitEstimator")
builder.add_edge("ProfitEstimator", END)

graph = builder.compile()

# 📊 Display Dashboard Function

This function takes the `final_state` from the LangGraph execution and displays the results in a structured format using `ipywidgets` and Markdown. It presents the suggested crops, fertilizer plan, estimated yield, market price, estimated profit, and the full AI-estimated summary.

In [ ]:
from IPython.display import Markdown

def display_dashboard(final_state):
    # Extract results
    crops = final_state.get("crop_suggestions", "N/A")
    ferts = final_state.get("fertilizer_plan", [])
    ferts_text = ", ".join(ferts) if isinstance(ferts, list) else ferts

    # Debug print for profit_summary
    profit_summary = final_state.get("ai_profit_analysis", "N/A")

    yield_est = final_state.get("estimated_yield", "N/A")
    price = final_state.get("market_price", "N/A")



    # Widgets
    crop_box = widgets.HTML(f"<b>🌾 Crops Suggested:</b><br>{crops}")
    fert_box = widgets.HTML(f"<b>💧 Fertilizer Plan:</b><br>{ferts_text}")
    yield_box = widgets.HTML(f"<b>📈 Estimated Yield:</b> {final_state.get('estimated_yield', 'N/A')}")
    price_box = widgets.HTML(f"<b>💰 Market Price:</b> {final_state.get('market_price', 'N/A')}")
    profit_box = widgets.HTML(f"<b>💸 Estimated Profit:</b> {final_state.get('estimated_profit', 'N/A')}")

    display(Markdown(f"### 💸 AI-Estimated Summary\n\n{profit_summary}"))

# 🚀 User Interface: Input Form and Workflow Execution

This cell creates an interactive form using `ipywidgets` for the user to input details about their land, soil, desired crop, budget, and region. It also defines the `on_submit_clicked` function, which is triggered when the submit button is pressed. This function clears the output, displays a processing message, invokes the LangGraph workflow with the user's input, and finally displays the results using the `display_dashboard` function.

In [ ]:
# 🎛️ Define User Input Widgets + Submit Button

import ipywidgets as widgets
from IPython.display import display, clear_output

# Define inputs
land_input = widgets.IntText(
    description="Land (m²)",
    placeholder="Enter land area in square meters"
)

soil_input = widgets.Dropdown(
    options=["Sandy", "Clay", "Loam", "Silty", "Black"],
    description="Soil"
)
crop_input = widgets.Combobox(
    placeholder='Type or choose crop (e.g., wheat, tomato, maize)',
    options=[
        "Wheat", "Rice", "Maize", "Groundnut", "Sunflower",
        "Tomato", "Onion", "Potato", "Chickpea", "Pigeon Pea",
        "Watermelon", "Muskmelon", "Cowpea", "Barley", "Sorghum"
    ],
    description="Crop"
)

budget_input = widgets.IntText(description="Budget (₹)")
region_input = widgets.Text(description="Region")

# Define button
submit_button = widgets.Button(description="Submit", button_style='success')

# Arrange form layout
input_form = widgets.VBox([
    land_input, soil_input, crop_input, budget_input, region_input, submit_button
])

# Global data holder
user_input_data = {}

# Define handler
def on_submit_clicked(_):

    global final_state

    if not all([land_input.value, soil_input.value, crop_input.value, budget_input.value, region_input.value]):
      print("❌ Please fill all fields before submitting.")
      return

    global user_input_data
    user_input_data = {
        "land": land_input.value,
        "soil": soil_input.value,
        "crop": crop_input.value,
        "budget": budget_input.value,
        "region": region_input.value
    }

    clear_output(wait=True)
    display(input_form)
    print("🧠 Running AgroGenie Multi-Agent Engine...")

    # Run LangGraph
    state = {"input": user_input_data}
    final_state = graph.invoke(state)

    # Show dashboard
    display_dashboard(final_state)

    # Show export + summary
    # display(export_btn, output_area)
    # display(ai_summary_box)

    # Optional: chart - Temporarily disabled as profit is in text summary
    # show_profit_chart(final_state)

# Link button
submit_button.on_click(on_submit_clicked)

# Show form
display(input_form)

# 🧠 Extracted AI Summary Recommendation

This cell extracts and displays the recommendation/comments section from the AI's profit analysis using `ipywidgets.HTML`. This provides a concise summary of the AI's insights and concerns.

In [ ]:
import markdown # Import the markdown library

ai_summary_text = final_state.get('ai_profit_analysis', 'N/A').split('Comments:')[-1].strip()

# Convert markdown to HTML
ai_summary_html = markdown.markdown(ai_summary_text)

ai_summary_box = widgets.HTML(
    value=f"<b>🧠 Recommendation:</b><br>{ai_summary_html}",
    layout=widgets.Layout(margin="20px 0 0 0")
)

display(ai_summary_box)

# 📂 Export AI Report as Text File

This cell provides functionality to export the complete AI-generated report as a text file. It defines an `export_report` function that formats the `final_state` data into a readable text report and creates a downloadable link using `IPython.display.FileLink`. A button (`export_btn`) is created to trigger the export process.

In [ ]:
# 📂 Export AI Report as Text File

from IPython.display import FileLink
import ipywidgets as widgets
import os
from google.colab import files

def export_report(final_state):
    report_path = "AgroGenie_Report.txt"
    with open(report_path, "w", encoding="utf-8") as f:
        f.write("🌱 AgroGenie Report\n\n")
        f.write("🔸 Region: " + final_state['input'].get('region', 'N/A') + "\n")
        f.write("🔸 Crop: " + final_state['input'].get('crop', 'N/A') + "\n")
        f.write("🔸 Soil: " + final_state['input'].get('soil', 'N/A') + "\n")
        f.write("🔸 Land: " + str(final_state['input'].get('land', 'N/A')) + " acres\n")
        f.write("🔸 Budget: ₹" + str(final_state['input'].get('budget', 'N/A')) + "\n\n")
        f.write("💡 Suggested Crops:\n" + str(final_state.get("crop_suggestions", "N/A")) + "\n\n")
        f.write("🧪 Fertilizer Plan:\n" + str(final_state.get("fertilizer_plan", "N/A")) + "\n\n")
        f.write("📈 Estimated Yield: " + str(final_state.get("estimated_yield", "N/A")) + "\n")
        f.write("💰 Market Price: " + str(final_state.get("market_price", "N/A")) + "\n")
        f.write("💸 Estimated Profit: " + str(final_state.get("estimated_profit", "N/A")) + "\n\n")
        f.write("🧠 AI Comments & Analysis:\n" + final_state.get("ai_profit_analysis", "N/A") + "\n")
    return report_path

# Button to trigger export
export_btn = widgets.Button(description="📄 Export Report", button_style="info")
output_area = widgets.Output()

def on_export_clicked(_):
    with output_area:
        output_area.clear_output()
        report_path = export_report(final_state)
        files.download(report_path)

export_btn.on_click(on_export_clicked)
display(export_btn, output_area)